# Demo for LumiSpy package working with AttoLight SEM CL data

This notebook shows:

- Loading a `HYPCard.bin` file correctly with the lumispy load function
- Plotting cathodoluminescence data in different ways
- Dealing with metadata
- Correcting for spectral defects

Import packages:

In [1]:
%matplotlib qt 
#%matplotlib inline
import lumispy as lum

### Loading HYPCard files

Use the `load_hypcard` function to load the AttoLight SEM specific files. State the relative path to the `HYPCard.bin` file.

In [2]:
cl_sem = lum.load_hypcard("DemoSample_192EF1E8-2AD5-4956-A718-BDAE8E3FAA0C/HYPCard.bin")

In [11]:
cl_sem.plot()

### Correction of spectral defects

Let's start for correcting the intrinsic shift caused by the misalignment of the grating with the spectrometer aperture centre. For the AttoLight in Cambridge, we have created a function to correct for it

In [12]:
cl_sem.correct_grating_shift()

Exception: You already corrected for the grating shift.

There are also corrections for the cosmic rays (pixels with sharp spikes):

In [13]:
#cl_sem.remove_cosmic_rays()

The edges of the scan get higher intensities, so they can be cropped. 
If you set the `replace` parameter to `True` the original CLSEMSpectrum object will be modified, if `False` a cropped copy of it will be created.

In [ ]:
#cl_sem.crop_edges(replace=False)

### Plotting data

Plot the corrected data:

In [3]:
cl_sem.plot()

Plot the data by the wavelength range:

In [7]:
cl_sem.T.plot()

### Working with the metadata

All the microscope & spectrometer parameters are saved in the metadata. It can be assessed via:

In [3]:
cl_sem.metadata

├── Acquisition_instrument
│   ├── SEM
│   │   ├── CCD
│   │   │   ├── binning = 1
│   │   │   └── channels = 1024
│   │   ├── FOV = 1317.35998
│   │   ├── Spectrometer
│   │   │   ├── central_wavelength = 760.0
│   │   │   └── grating = 150.0
│   │   ├── resolution_x = 128
│   │   └── resolution_y = 128
│   └── acquisition_system = cambridge_attolight
├── General
│   └── title = DemoSample
└── Signal
    ├── binned = False
    └── signal_type = cl_sem_spectrum

You can add any other parameter manually using:

In [8]:
cl_sem.metadata.set_item('Acquisition_instrument.SEM.acceleration_voltage', 3)

In [9]:
cl_sem.metadata

├── Acquisition_instrument
│   ├── SEM
│   │   ├── CCD
│   │   │   ├── binning = 1
│   │   │   └── channels = 1024
│   │   ├── FOV = 1317.35998
│   │   ├── Spectrometer
│   │   │   ├── central_wavelength = 760.0
│   │   │   └── grating = 150.0
│   │   ├── acceleration_voltage = 3
│   │   ├── resolution_x = 128
│   │   └── resolution_y = 128
│   └── acquisition_system = cambridge_attolight
├── General
│   └── title = DemoSample
└── Signal
    ├── binned = False
    ├── grating_corrected = True
    └── signal_type = cl_sem_spectrum

In [11]:
cl_sem.plot()